In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')

Mounted at /content/drive


In [ ]:
!pip install openai

In [ ]:
import pandas as pd

In [ ]:
!ls "ece1786 project/Progress Report"

small_ds.csv  small_ds.gsheet  Untitled0.ipynb


In [ ]:
import pandas as pd
file = pd.read_csv('/content/drive/MyDrive/ece1786 project/Progress Report/small_ds.csv')
file['Answer'].apply(len)

0      5
1      4
2      4
3     10
4      4
5     10
6      4
7      3
8      5
9      4
10    11
11     5
12     7
13     4
14     3
15     6
16     3
17     7
18     3
19     6
20     3
21     4
22     7
23     5
24    11
25     4
26     5
27     3
28     4
29    10
30     4
31    10
32     4
33     4
34     5
Name: Answer, dtype: int64

In [ ]:
def small_dataset(file='/content/drive/MyDrive/ece1786 project/Progress Report/small_ds.csv'):
  # parse data from https://www.xwordinfo.com/Crossword?date=11/10/2023
  df = pd.read_csv(file)
  df['first_letter'] = df['Answer'].str[0]
  df['num_letters'] = df['Answer'].apply(len)
  return df

#CoT
### 1. Providing first letter.

In [ ]:
import pandas as pd
import openai

In [ ]:
def fill_prompt(row):
  '''
  row: a row in dataframe
  '''
  prompt_template = f"What is this word given the clue \"{row['Clue']}\", \
  the word has {row['num_letters']} letters, and it starts with the letter '{row['first_letter']}'. \
  Let's think step by step, when you see the clue \"{row['Clue']}\", what words would it make you think of? \
  If the clue is a noun in plural form, the answer also needs to be in plural form. \
  Output a list of 10 words you can think of, ideally they should start with '{row['first_letter']}'. \
  Then, from this list of words, say which one(s) start with the letter '{row['first_letter']}' and have {row['num_letters']} letters.\
  In the last sentence of your response, state the word you think makes the most sense, like so: \
  The word is: <your answer here>."

  return prompt_template

def get_gpt_guess(row):
  raw_word = row['gpt_guess'].split("is:")[-1]
  reg = raw_word.split(".")[0]
  reg = reg.replace('"', '')
  reg = reg.replace('\'', '')
  reg = reg.replace(' ', '')
  return reg

def is_puzzle_ans(row):

  return row['Answer'].lower() == row['gpt_parsed_answer'].lower()


In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('OPEN_AI_KEY')
openai.api_key = openai_api_key

In [ ]:
prompt_col = "prompt"
row = small_dataset_cond[prompt_col]

In [ ]:
# This code is for v1 of the openai package: pypi.org/project/openai
from openai import OpenAI
client = OpenAI(api_key=openai.api_key)

def get_gpt_resp(row):
  response = client.chat.completions.create(
    model="gpt-4",
    messages=[
      {
        "role": "user",
        "content": f"{row['prompt']}"
      }
    ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

In [ ]:
small_dataset= small_dataset()
small_dataset['prompt'] = small_dataset.apply(fill_prompt, axis=1)

In [ ]:
small_dataset['gpt_guess'] = small_dataset.apply(get_gpt_resp, axis=1)

In [ ]:
small_dataset.to_csv('/content/drive/MyDrive/ece1786 project/Progress Report/small_ds_gpt.csv')

In [ ]:
small_dataset['gpt_parsed_answer'] = small_dataset.apply(get_gpt_guess, axis=1)
small_dataset

,Clue,Answer,first_letter,num_letters,prompt,gpt_guess,gpt_parsed_answer
0,Not up to the job,UNFIT,U,5,"What is this word given the clue ""Not up to th...",Here are 10 words that start with 'U' that com...,Unfit
1,"""Butt out!,"" in brief",MYOB,M,4,"What is this word given the clue """"Butt out!,""...","The clue """"Butt out!,"" in brief"" is indicative...",Mind
2,Up to the job,ABLE,A,4,"What is this word given the clue ""Up to the jo...","The clue ""Up to the job"" makes me think of wor...",Able
3,Subtly damaging,PERNICIOUS,P,10,"What is this word given the clue ""Subtly damag...","Some words that come to mind from the clue ""su...",pernicious
4,Copy cat?,MEOW,M,4,"What is this word given the clue ""Copy cat?"", ...","The clue ""Copy cat?"" might make one think of w...",meme
5,"Drives around in circles, perhaps",DOESDONUTS,D,10,"What is this word given the clue ""Drives aroun...",1. Distracts\n2. Driveways\n3. Directives\n4. ...,Diversions
6,Pharmaceutical plant,ALOE,A,4,"What is this word given the clue ""Pharmaceutic...","Some words that come to mind when I see ""Pharm...",Aloe
7,Unified,ONE,O,3,"What is this word given the clue ""Unified"", ...",Some words that start with 'O' and have to do ...,One
8,Low parts of an opera,BASSI,B,5,"What is this word given the clue ""Low parts of...",The list of 'B' words related to opera: \n\n1....,Basses
9,Comic strip with a teenage protagonist,ZITS,Z,4,"What is this word given the clue ""Comic strip ...","Some words that come to mind include Zits, Zan...",Zits


In [ ]:
small_dataset['is_correct'] = small_dataset.apply(is_puzzle_ans, axis=1)

In [ ]:
small_dataset.shape

(35, 8)

In [ ]:
small_dataset['is_correct'].value_counts(normalize=True)
# score for correct is 0.6

True     0.6
False    0.4
Name: is_correct, dtype: float64

In [ ]:
small_dataset.to_csv('/content/drive/MyDrive/ece1786 project/Progress Report/small_ds_gpt_cleaned.csv')

If we only look at the ones where the answer is a one-word answer and not an abbreviation, the success rate is much higher.

In [ ]:
def check_condition(row):
  '''
  check which words are: more than one word, is a abbreviation, is a foreign language, is a suffix/prefix
  we want to exclude them from the success rate calculation
  '''
  return (row['is_one_word']=='T') and (row['is_english']=='T') and (row['is_not_abbrev']=='T') and (row['is_not_suffix_prefix']=='T')

def meet_condition_and_correct(df):
  # calculate, for the ones that meet condition, how many are correct, how many are wrong
  new_df = df.drop(df[df.condition=='False'].index)
  return new_df


In [ ]:
small_dataset_cond = pd.read_csv('/content/drive/MyDrive/ece1786 project/Progress Report/small_ds_gpt_cleaned_conditions.csv')

In [ ]:
small_dataset_cond['condition'] = small_dataset_cond.apply(check_condition, axis=1)


,Unnamed: 0,Clue,Answer,is_one_word,is_english,is_not_abbrev,is_not_suffix_prefix,first_letter,num_letters,prompt,gpt_guess,gpt_parsed_answer,is_correct,condition
0,0,Not up to the job,UNFIT,T,T,T,T,U,5,"What is this word given the clue ""Not up to th...",Here are 10 words that start with 'U' that com...,Unfit,True,True
1,1,"""Butt out!,"" in brief",MYOB,F,T,F,T,M,4,"What is this word given the clue """"Butt out!,""...","The clue """"Butt out!,"" in brief"" is indicative...",Mind,False,False
2,2,Up to the job,ABLE,T,T,T,T,A,4,"What is this word given the clue ""Up to the jo...","The clue ""Up to the job"" makes me think of wor...",Able,True,True
3,3,Subtly damaging,PERNICIOUS,T,T,T,T,P,10,"What is this word given the clue ""Subtly damag...","Some words that come to mind from the clue ""su...",pernicious,True,True
4,4,Copy cat?,MEOW,T,T,T,T,M,4,"What is this word given the clue ""Copy cat?"", ...","The clue ""Copy cat?"" might make one think of w...",meme,False,True
5,5,"Drives around in circles, perhaps",DOESDONUTS,F,T,F,T,D,10,"What is this word given the clue ""Drives aroun...",1. Distracts\n2. Driveways\n3. Directives\n4. ...,Diversions,False,False
6,6,Pharmaceutical plant,ALOE,T,T,T,T,A,4,"What is this word given the clue ""Pharmaceutic...","Some words that come to mind when I see ""Pharm...",Aloe,True,True
7,7,Unified,ONE,T,T,T,T,O,3,"What is this word given the clue ""Unified"", ...",Some words that start with 'O' and have to do ...,One,True,True
8,8,Low parts of an opera,BASSI,T,T,T,T,B,5,"What is this word given the clue ""Low parts of...",The list of 'B' words related to opera: \n\n1....,Basses,False,True
9,9,Comic strip with a teenage protagonist,ZITS,T,T,T,T,Z,4,"What is this word given the clue ""Comic strip ...","Some words that come to mind include Zits, Zan...",Zits,True,True


In [ ]:
small_dataset_cond = small_dataset_cond.drop(small_dataset_cond[small_dataset_cond.condition==False].index)

In [ ]:
small_dataset_cond['is_correct'].value_counts(True)
# if we exclude the words that don't meet the condition - is one word, not an abbreviation, not a foreign language, not a prefix/suffix,
# the success rate is 80.8%
# therefore need to modify the prompt to account for these cases

True     0.807692
False    0.192308
Name: is_correct, dtype: float64

In [ ]:
def prompt_to_gen_clue(row):
  '''
  row: a row in dataframe
  '''
  prompt_template = f"Given the word \"{row['gpt_parsed_answer']}\", \
  generate another crossword clue that is not \"{row['Clue']}\". Output your clue in the format like so,\
  Another clue is: <your clue here>."

  return prompt_template

In [ ]:
small_dataset_cond = pd.read_csv('/content/drive/MyDrive/ece1786 project/Progress Report/small_ds_gpt_cleaned_conditions.csv')

In [ ]:
small_dataset_cond["clue_prompt"] = small_dataset_cond.apply(prompt_to_gen_clue, axis=1)

In [ ]:
def get_gpt_clue(row, prompt_col):
  if row['is_correct'] is False:
    return "No clue available."
  else:
    response = client.chat.completions.create(
      model="gpt-4",
      messages=[
        {
          "role": "user",
          "content": f"{row[prompt_col]}"
        }
      ],
      temperature=1,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response.choices[0].message.content

In [ ]:
small_dataset_cond["gpt_clue"] = small_dataset_cond.apply(get_gpt_clue, prompt_col='clue_prompt', axis=1)

In [ ]:
small_dataset_cond

,Unnamed: 0,Clue,Answer,is_one_word,is_english,is_not_abbrev,is_not_suffix_prefix,first_letter,num_letters,prompt,gpt_guess,gpt_parsed_answer,is_correct,clue_prompt,gpt_clue
0,0,Not up to the job,UNFIT,T,T,T,T,U,5,"What is this word given the clue ""Not up to th...",Here are 10 words that start with 'U' that com...,Unfit,True,"Given the word ""Unfit"", generate another cro...","Another clue is: ""Lacking in physical condition"""
1,1,"""Butt out!,"" in brief",MYOB,F,T,F,T,M,4,"What is this word given the clue """"Butt out!,""...","The clue """"Butt out!,"" in brief"" is indicative...",Mind,False,"Given the word ""Mind"", generate another cros...",No clue available.
2,2,Up to the job,ABLE,T,T,T,T,A,4,"What is this word given the clue ""Up to the jo...","The clue ""Up to the job"" makes me think of wor...",Able,True,"Given the word ""Able"", generate another cros...","Another clue is: ""Capable or competent""."
3,3,Subtly damaging,PERNICIOUS,T,T,T,T,P,10,"What is this word given the clue ""Subtly damag...","Some words that come to mind from the clue ""su...",pernicious,True,"Given the word ""pernicious"", generate anothe...","Another clue is: ""Harmful in a covert or gradu..."
4,4,Copy cat?,MEOW,T,T,T,T,M,4,"What is this word given the clue ""Copy cat?"", ...","The clue ""Copy cat?"" might make one think of w...",meme,False,"Given the word ""meme"", generate another cros...",No clue available.
5,5,"Drives around in circles, perhaps",DOESDONUTS,F,T,F,T,D,10,"What is this word given the clue ""Drives aroun...",1. Distracts\n2. Driveways\n3. Directives\n4. ...,Diversions,False,"Given the word ""Diversions"", generate anothe...",No clue available.
6,6,Pharmaceutical plant,ALOE,T,T,T,T,A,4,"What is this word given the clue ""Pharmaceutic...","Some words that come to mind when I see ""Pharm...",Aloe,True,"Given the word ""Aloe"", generate another cros...","Another clue is: ""Succulent used in skincare"""
7,7,Unified,ONE,T,T,T,T,O,3,"What is this word given the clue ""Unified"", ...",Some words that start with 'O' and have to do ...,One,True,"Given the word ""One"", generate another cross...","Another clue is: ""Singleton digit"""
8,8,Low parts of an opera,BASSI,T,T,T,T,B,5,"What is this word given the clue ""Low parts of...",The list of 'B' words related to opera: \n\n1....,Basses,False,"Given the word ""Basses"", generate another cr...",No clue available.
9,9,Comic strip with a teenage protagonist,ZITS,T,T,T,T,Z,4,"What is this word given the clue ""Comic strip ...","Some words that come to mind include Zits, Zan...",Zits,True,"Given the word ""Zits"", generate another cros...","Another clue is: ""Common teenage skin blemishes"""


In [ ]:
small_dataset_cond.to_csv("/content/drive/MyDrive/ece1786 project/Progress Report/small_dataset_gpt_clue.csv")